在本教程中，你将了解什么是数据泄漏（data leakage）以及如何避免它。如果不了解如何规避，泄漏会频繁出现，并以隐蔽且危险的方式破坏模型。因此，这是数据科学实战中最重要的概念之一。

# 引言

**数据泄漏**（或简称**泄漏**）指的是：训练数据中包含了关于目标变量的信息，但在实际预测时并不会获得到这些信息。结果是训练集（甚至验证集）表现很高，但模型在生产环境中表现很差。

换句话说，泄漏会让模型在你用它做决策之前看起来很准确；一旦真正用于决策，模型就会变得很不准。

泄漏主要分为两类：**目标泄漏（target leakage）**和**训练-测试污染（train-test contamination）**。

### 目标泄漏（Target leakage）

当你的特征中包含在预测时点不可用的信息时，就发生了**目标泄漏**。思考目标泄漏时，重点是数据可用性的“时间/时间顺序”，而不仅仅是某个特征是否有助于预测。

看一个例子。假设我们要预测谁会得肺炎。原始数据的前几行如下：

| got_pneumonia | age | weight |  male | took_antibiotic_medicine | ... |
|:-------------:|:---:|:------:|:-----:|:------------------------:|-----|
|     False     |  65 |   100  | False |           False          | ... |
|     False     |  72 |   130  |  True |           False          | ... |
|      True     |  58 |   100  | False |           True           | ... |

人们是在得了肺炎之后才服用抗生素。原始数据表明这些列之间关联很强，但 `took_antibiotic_medicine` 往往是在 `got_pneumonia` 的值已确定之后才被更新。这就是目标泄漏。

模型会“学到”：凡是 `took_antibiotic_medicine=False` 的人都没有肺炎。由于验证数据与训练数据同源，这一模式会在验证中重复出现，使得验证（或交叉验证）分数看起来很高。

但在真实世界部署时就会很不准确——因为即便患者将来会得肺炎，在我们需要预测其未来健康状况之时，他们尚未服用抗生素。

为了避免这种泄漏，任何在目标值“发生之后”才会被更新（或创建）的变量都应被排除。

![tut7_leakydata](https://storage.googleapis.com/kaggle-media/learn/images/y7hfTYe.png)

### 训练-测试污染（Train-Test Contamination）

另一类泄漏发生在你没有严格区分训练数据与验证数据的时候。

回顾：验证的目的是评估模型在“未见过的数据”上的表现。如果验证数据影响了预处理行为，就会以隐蔽方式破坏这一过程，这被称为**训练-测试污染**。

例如，如果你在调用 [`train_test_split()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) 之前就进行了预处理（如先拟合了缺失值插补器），会怎样？结果是模型在验证集上得分很好，让你过度自信，但上线后表现糟糕。

归根结底，你在“如何做预测”的过程中使用了来自验证/测试的数据，因此模型可能在那部分数据上表现好，却无法泛化到新数据。做更复杂的特征工程时，这个问题会更隐蔽、更危险。

如果你采用简单的训练-测试划分作为验证方式，就必须把验证集排除在任何形式的“拟合”之外，包括预处理步骤的拟合。若使用 scikit-learn 的管道，这会容易许多；而在交叉验证中，更要把预处理放进管道里！

# 示例

本示例演示一种检测并移除目标泄漏的方法。

我们将使用信用卡申请数据集，并跳过基础的读取/清洗代码。最终结果是：每个申请的信息保存在 DataFrame `X` 中，我们要用它来预测 Series `y` 中的“是否被批准”。

In [ ]:

import pandas as pd

# Read the data
data = pd.read_csv('../input/aer-credit-card-data/AER_credit_card_data.csv', 
                   true_values = ['yes'], false_values = ['no'])

# Select target
y = data.card

# Select predictors
X = data.drop(['card'], axis=1)

print("Number of rows in the dataset:", X.shape[0])
X.head()

由于数据集较小，我们使用交叉验证来获得更可靠的模型质量估计。

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Since there is no preprocessing, we don't need a pipeline (used anyway as best practice!)
my_pipeline = make_pipeline(RandomForestClassifier(n_estimators=100))
cv_scores = cross_val_score(my_pipeline, X, y, 
                            cv=5,
                            scoring='accuracy')

print("Cross-validation accuracy: %f" % cv_scores.mean())

有经验后你会发现：模型“98% 准确率”非常罕见。虽然可能发生，但足够罕见到值得我们进一步检查是否存在目标泄漏。

下面是数据字典（在数据标签页也能看到）：

 - **`card`**：是否批准信用卡申请（1=批准，0=未批准）
 - **`reports`**：不良记录报告数量（严重类）
 - **`age`**：年龄（年，带 1/12 年的小数）
 - **`income`**：年收入（除以 10,000）
 - **`share`**：月信用卡支出 / 年收入
 - **`expenditure`**：平均月信用卡支出
 - **`owner`**：是否自有住房（1=自有，0=租住）
 - **`selfempl`**：是否自雇（1=自雇，0=否）
 - **`dependents`**：1 + 家属数量
 - **`months`**：在当前住址的月数
 - **`majorcards`**：持有的主流信用卡数量
 - **`active`**：活跃信用账户数量

有些变量看起来可疑。例如，**`expenditure`** 指的是这张卡的消费，还是申请前使用的其他卡的消费？

此时，做一些基础对比很有帮助：

In [ ]:
expenditures_cardholders = X.expenditure[y]
expenditures_noncardholders = X.expenditure[~y]

print('Fraction of those who did not receive a card and had no expenditures: %.2f' \
      %((expenditures_noncardholders == 0).mean()))
print('Fraction of those who received a card and had no expenditures: %.2f' \
      %(( expenditures_cardholders == 0).mean()))

如上所示，未获批信用卡的人“全部没有消费”，而获批者中只有 2% 没有消费。模型准确率高就不足为奇了。但这似乎正是目标泄漏：`expenditure` 很可能指的是“申请这张卡后的消费”。

由于 **`share`** 部分由 **`expenditure`** 决定，它也应当被排除。**`active`** 与 **`majorcards`** 虽然不那么明确，但从描述看也值得警惕。多数情况下，如果无法联系到数据创建者进一步确认，宁可保守一些。

不含目标泄漏的建模流程如下：

In [ ]:
# Drop leaky predictors from dataset
potential_leaks = ['expenditure', 'share', 'active', 'majorcards']
X2 = X.drop(potential_leaks, axis=1)

# Evaluate the model with leaky predictors removed
cv_scores = cross_val_score(my_pipeline, X2, y, 
                            cv=5,
                            scoring='accuracy')

print("Cross-val accuracy: %f" % cv_scores.mean())

该准确率明显更低，可能让人失望。然而，用在新的申请上时，我们大约能期望 80% 的正确率；而存在泄漏的模型尽管交叉验证分数更高，实际表现很可能更糟。

# 结论
在许多数据科学应用中，数据泄漏可能造成数百万美元级别的损失。严格区分训练与验证数据可以防止训练-测试污染，使用管道有助于落实这种隔离。同样，谨慎+常识+数据探索的组合有助于识别目标泄漏。

# 接下来做什么？

这些概念可能仍显抽象。通过 **[这道练习](https://www.kaggle.com/kernels/fork/3370270)** 中的示例多加思考，提升你识别目标泄漏与训练-测试污染的能力！

---




*有问题或想交流？欢迎访问[课程讨论区](https://www.kaggle.com/learn/intermediate-machine-learning/discussion)与其他学习者交流。*